# gold_churn_predictions (Daily Job Final)

이 노트북은 `dlt_gold_user_behavior_snapshot`(T-1 스냅샷)을 기반으로
하루 1회 Churn 예측을 수행하여 `gold_churn_predictions` 테이블에 **해당 날짜(event_date)만** 갱신합니다.

- 기본 실행: `score_date` 미지정 → **T-1(어제)** 기준(없으면 T-1 이하 최신 날짜로 fallback)
- 백필(backfill): `score_date=YYYY-MM-DD` 지정 → 해당 날짜만 재계산/갱신
- 저장 방식: `replaceWhere`로 **event_date=score_date** 파티션만 overwrite → 과거 데이터는 유지(누적)

> 모델 로드는 팀 기존 방식 유지: `models:/...@latest`


In [0]:
%pip install -U mlflow xgboost scikit-learn
dbutils.library.restartPython()

#### 로직 변경 모델 적용 버전(2026-02-24)
churn_reason_encoded 제거

In [0]:
import mlflow.xgboost
from pyspark.sql import functions as F
import pandas as pd
import numpy as np

# =========================
# 0) Config
# =========================
CATALOG = "signalcraft_databricks"
SCHEMA  = "default"

SNAPSHOT_TABLE = f"{CATALOG}.{SCHEMA}.dlt_gold_user_behavior_snapshot"
OUT_TABLE      = f"{CATALOG}.{SCHEMA}.gold_churn_predictions"

# 모델 로드 (팀 기존 방식 유지)
mlflow.set_registry_uri("databricks-uc")
model_name = f"{CATALOG}.{SCHEMA}.churn_predictor"
model_uri  = f"models:/{model_name}@latest"   # ✅ 유지

# =========================
# 1) score_date 결정 (기본: T-1)
# =========================
dbutils.widgets.text("score_date", "")
param_date = dbutils.widgets.get("score_date").strip()

# 원칙적으로 T-1(어제) 사용
t1 = spark.sql("SELECT date_sub(current_date(), 1) AS d").first()["d"]

snapshot_df = spark.table(SNAPSHOT_TABLE)

if param_date:
    score_date = param_date
else:
    # T-1이 없을 수 있으니 "T-1 이하"에서 가장 최신 날짜로 fallback
    score_date = (snapshot_df
                  .filter(F.col("event_date") <= F.lit(t1))
                  .agg(F.max("event_date").alias("max_d"))
                  .first()["max_d"])
    if score_date is None:
        score_date = t1

print(f"📅 추론 기준일(score_date): {score_date}")

# =========================
# 2) 모델 로드
# =========================
model = mlflow.xgboost.load_model(model_uri)
print(f"✅ Model loaded: {model_uri}")

# =========================
# 3) 대상 데이터 로드 (해당 날짜 & Active)
# =========================
required_cols = {
    "user_id", "event_date", "is_active",
    "daily_watch_time_min", "watch_time_7d_min", "watch_time_30d_min",
    "active_days_7", "active_days_30", "days_since_last_login",
    "churn_risk_level"  # risk_level_encoded를 만들기 위해 필요
}
missing = sorted(list(required_cols - set(snapshot_df.columns)))
if missing:
    raise ValueError(f"Snapshot table missing required columns: {missing}")

features_df = (snapshot_df
               .filter((F.col("event_date") == F.lit(score_date)) & (F.col("is_active") == 1)))

target_cnt = features_df.count()
print(f"👥 실질 추론 대상(Active): {target_cnt}명")

if target_cnt == 0:
    raise RuntimeError(f"No active users found for score_date={score_date}. Job will fail fast.")

# =========================
# 4) 피처 엔지니어링 + Pandas 변환
# =========================
inference_ready_df = (features_df
    .withColumn("watch_time_ratio", F.col("watch_time_7d_min") / (F.col("watch_time_30d_min") / 4 + F.lit(1)))
    .withColumn("active_days_ratio", F.col("active_days_7") / (F.col("active_days_30") / 4 + F.lit(1)))
)

# churn_reason은 더 이상 사용하지 않으므로 select에서 제외
pdf = inference_ready_df.select(
    "user_id", "event_date",
    "daily_watch_time_min", "watch_time_7d_min", "watch_time_30d_min",
    "active_days_7", "active_days_30", "days_since_last_login",
    "watch_time_ratio", "active_days_ratio",
    "churn_risk_level" # churn_reason 제거됨
).toPandas()

# Pandas 변환 후 수치형 데이터 결측치 보정 (Inference 안정성)
numeric_cols = [
    "daily_watch_time_min", "watch_time_7d_min", "watch_time_30d_min",
    "active_days_7", "active_days_30", "days_since_last_login",
    "watch_time_ratio", "active_days_ratio"
]
pdf[numeric_cols] = pdf[numeric_cols].fillna(0) # 결측치는 0으로 채움

# =========================
# 5) 범주형 인코딩 (학습 시 정의 순서 고정)
# =========================
from sklearn.preprocessing import LabelEncoder

# churn_risk_level
le_risk = LabelEncoder()
le_risk.fit(['Active', 'Soft Churn', 'Dormant', 'Churned'])
if pdf["churn_risk_level"].isna().any():
    raise RuntimeError("Found null churn_risk_level values in snapshot. Please fix upstream data.")
pdf["risk_level_encoded"] = le_risk.transform(pdf["churn_risk_level"])

# =========================
# 6) 피처 컬럼 정의 (학습 순서와 동일해야 함)
# =========================
feature_cols = [
    'daily_watch_time_min', 
    'watch_time_7d_min', 
    'watch_time_30d_min',
    'active_days_7', 
    'active_days_30', 
    'days_since_last_login',
    'watch_time_ratio', 
    'active_days_ratio',
    'risk_level_encoded' # churn_reason_encoded는 여기서 삭제됨
]

# 결측 처리(최소 안전장치): 숫자 결측은 0으로 채움 (필요시 팀 방식으로 변경)
pdf[feature_cols] = pdf[feature_cols].replace([np.inf, -np.inf], np.nan)
pdf[feature_cols] = pdf[feature_cols].fillna(0)

# =========================
# 6.1) 피처 정합성 강제 확인 (에러 방지 핵심)
# =========================
# 학습 당시 모델이 기대하는 피처 이름들을 가져옵니다.
try:
    expected_features = model.get_booster().feature_names
    print(f"📋 Model expects: {expected_features}")
except:
    # 만약 위 코드가 작동하지 않는다면 수동 정의된 리스트 사용
    expected_features = feature_cols

# 모델이 기대하는 피처가 pdf에 없는 경우 0으로 생성
for col in expected_features:
    if col not in pdf.columns:
        print(f"⚠️ Missing field found: {col}. Creating with default 0.")
        pdf[col] = 0

# 모델이 기대하는 피처 '순서'대로 데이터프레임 재정렬 (매우 중요)
final_pdf = pdf[expected_features]

# =========================
# 7) 추론 및 probability 계산
# =========================
pdf['churn_probability'] = model.predict_proba(pdf[feature_cols])[:, 1]

# =========================
# 8) Probability Banding (팀 기존 기준 유지)
# =========================
conditions = [
    (pdf['churn_probability'] < 0.44),
    (pdf['churn_probability'] >= 0.44) & (pdf['churn_probability'] < 0.51),
    (pdf['churn_probability'] >= 0.51) & (pdf['churn_probability'] < 0.55),
    (pdf['churn_probability'] >= 0.55)
]

choices = ['Low', 'Mid', 'High', 'Critical']
pdf['probability_band'] = np.select(conditions, choices, default='Unknown')

# =========================
# 9) 결과 저장 (해당 날짜만 overwrite → 히스토리 유지)
# =========================
result_columns = ['user_id', 'event_date', 'churn_probability', 'probability_band']
result_spark_df = spark.createDataFrame(pdf[result_columns])

(result_spark_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"event_date = '{score_date}'")
    .saveAsTable(OUT_TABLE)
)

# =========================
# 10) 요약 로그
# =========================
print("✅ band distribution")
print(pdf['probability_band'].value_counts())
print(f"✅ {score_date} 기준 추론 완료! → {OUT_TABLE}")


In [0]:
predictions_df = spark.table("signalcraft_databricks.default.gold_churn_predictions").where(f"event_date = '{score_date}'")
predictions_pdf = predictions_df.toPandas()

# 현재 활성 유저들의 실제 점수 분포 확인
predictions_stats = predictions_pdf['churn_probability'].describe(percentiles=[.25, .5, .75, .9])
print(predictions_stats)

In [0]:
# XGBoost 모델인 경우
try:
    # 학습 시 저장된 피처 이름 리스트 출력
    expected_features = model.get_booster().feature_names
    print(f"현재 모델이 기대하는 피처 리스트 ({len(expected_features)}개):")
    print(expected_features)
except Exception as e:
    print(f"피처 이름을 가져올 수 없습니다: {e}")